In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/bs140513_032310.csv')
data

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0
...,...,...,...,...,...,...,...,...,...,...
594638,179,'C1753498738','3','F','28007','M1823072687','28007','es_transportation',20.53,0
594639,179,'C650108285','4','F','28007','M1823072687','28007','es_transportation',50.73,0
594640,179,'C123623130','2','F','28007','M349281107','28007','es_fashion',22.44,0
594641,179,'C1499363341','5','M','28007','M1823072687','28007','es_transportation',14.46,0


Уберем лишние кавычки.

In [ ]:
data.columns

Index(['step', 'customer', 'age', 'gender', 'zipcodeOri', 'merchant',
       'zipMerchant', 'category', 'amount', 'fraud'],
      dtype='object')

In [ ]:
cols_to_edit = ['customer', 'age', 'gender', 'zipcodeOri', 'merchant', 'zipMerchant', 'category'] 
     
data[cols_to_edit] = data[cols_to_edit].replace({"'":""}, regex=True)

In [ ]:
data.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,C1093826151,4,M,28007,M348934600,28007,es_transportation,4.55,0
1,0,C352968107,2,M,28007,M348934600,28007,es_transportation,39.68,0
2,0,C2054744914,4,F,28007,M1823072687,28007,es_transportation,26.89,0
3,0,C1760612790,3,M,28007,M348934600,28007,es_transportation,17.25,0
4,0,C757503768,5,M,28007,M348934600,28007,es_transportation,35.72,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594643 entries, 0 to 594642
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   step         594643 non-null  int64  
 1   customer     594643 non-null  object 
 2   age          594643 non-null  object 
 3   gender       594643 non-null  object 
 4   zipcodeOri   594643 non-null  object 
 5   merchant     594643 non-null  object 
 6   zipMerchant  594643 non-null  object 
 7   category     594643 non-null  object 
 8   amount       594643 non-null  float64
 9   fraud        594643 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 45.4+ MB


In [ ]:
data['zipcodeOri'].unique(), data['zipMerchant'].unique()

(array(['28007'], dtype=object), array(['28007'], dtype=object))

Поскольку все данные о ZIP-коде одинаковые, можно их удалить.

In [ ]:
data.drop(columns=['zipcodeOri', 'zipMerchant'], inplace=True)

Разделим данные на две части: выделим последние 5000 транзакций в отдельный блок, чтобы можно было оценить алгоритм.

In [ ]:
eval_data = data.iloc[-5000:].copy()

In [ ]:
inv_data = data.iloc[:-5000].copy()

Посмотрим на каждую категорию (клиент, возраст, пол и т.д.) и найдём для каждого пункта в этой категории, какой процент от общего числа транзакций составляют мошеннические, может, обнаружим что-то подозрительное.

In [ ]:
item_names = ['customer', 'age', 'gender', 'merchant', 'category', 'amount']
items = dict()
for name in item_names:
  temp_df = inv_data.groupby(name)['fraud'].agg(['sum', 'count'])
  temp_df['rate'] = temp_df['sum']/temp_df['count']
  items[name] = temp_df.sort_values(by='rate', ascending=False)


Посмотрим сначала на категории "пол" и "возраст".

In [ ]:
items['gender']

,sum,count,rate
gender,,,
F,4719,321803,0.014664
M,2418,266160,0.009085
E,7,1168,0.005993
U,0,512,0.000000


У женщин доля мошеннических транзакций немного выше, чем у мужчин, но не настолько, чтобы один пол клиента мог вызвать подозрения. Транзакций, где пол неизвестен, слишком мало, чтобы можно было сделать на основании этого какие-то выводы; то же самое можно сказать про организации - кажется, что такие транзакции безопаснее, но их всё-таки слишком мало для определённых выводов.

In [ ]:
items['age']

,sum,count,rate
age,,,
0,47,2426,0.019373
4,1401,108105,0.012960
2,2329,185757,0.012538
3,1746,145869,0.011970
1,678,57640,0.011763
5,678,62120,0.010914
6,258,26558,0.009715
U,7,1168,0.005993


В отношении возраста у всех групп примерно одинаковая доля мошеннических транзакций, за исключением категории "возраст неизвестен", но таких данных в целом мало.

Можно сказать, что возраст и пол не могут служить триггерами.

Теперь посмотрим на клиентов.


In [ ]:
items['customer']

,sum,count,rate
customer,,,
C910454738,86,91,0.945055
C2004941826,119,126,0.944444
C381007057,87,98,0.887755
C617723960,39,44,0.886364
C1572610482,89,101,0.881188
...,...,...,...
C1389003663,0,140,0.000000
C2020967123,0,154,0.000000
C1388726904,0,176,0.000000


Видно, что есть как клиенты без мошеннических транзакции, так и клиенты, у которых большинство транзакций - мошеннические.

Клиентов можно разделить на три группы риска:

- те, у кого доля мошеннических транзакций составляет более 60%
- те, у кого доля мошеннических транзакций составляет более 20%, но менее 60%
- те, у кого доля мошеннических транзакций не более 20%

In [ ]:
def risk_group(df, column, low_limit, hi_limit):
  return df[(df['rate'] > low_limit) & (df['rate'] <= hi_limit)].reset_index()[column].to_list()

In [ ]:
customer_highrisk = risk_group(items['customer'], 'customer', 0.6, 1)
customer_medrisk = risk_group(items['customer'], 'customer', 0.2, 0.6)
customer_lowrisk = risk_group(items['customer'], 'customer', 0, 0.2)

Теперь посмотрим на продавцов.

In [ ]:
items['merchant']

,sum,count,rate
merchant,,,
M1294758098,182,189,0.962963
M3697346,282,300,0.940000
M1873032707,214,248,0.862903
M732195782,516,606,0.851485
M980657600,1466,1763,0.831537
M1353266412,62,76,0.815789
M857378720,92,122,0.754098
M2080407379,36,48,0.750000
M2011752106,164,242,0.677686


Также поделим их на группы риска (с критериями, аналогичными применёнными к клиентам).

In [ ]:
merchant_highrisk = risk_group(items['merchant'], 'merchant', 0.6, 1)
merchant_medrisk = risk_group(items['merchant'], 'merchant', 0.2, 0.6)
merchant_lowrisk = risk_group(items['merchant'], 'merchant', 0, 0.2)

Посмотрим на категории транзакций.

In [ ]:
items['category']

,sum,count,rate
category,,,
es_leisure,464,489,0.948875
es_travel,576,726,0.793388
es_sportsandtoys,1976,3985,0.495859
es_hotelservices,542,1729,0.313476
es_otherservices,224,904,0.247788
es_home,300,1975,0.151899
es_health,1684,15979,0.105388
es_tech,156,2355,0.066242
es_wellnessandbeauty,708,14999,0.047203


Также разобьём их на группы, но в данной ситуации немного изменим критерии, чтобы категория "es_sportsandtoys" тоже попадала в группу высокого риска.

In [ ]:
category_highrisk = risk_group(items['category'], 'category', 0.49, 1)
category_medrisk = risk_group(items['category'], 'category', 0.2, 0.49)
category_lowrisk = risk_group(items['category'], 'category', 0, 0.2)

Посмотрим на суммы транзакций.

In [ ]:
items['amount'].sort_values('sum', ascending=False)

,sum,count,rate
amount,,,
159.82,4,5,0.80
245.34,3,3,1.00
126.51,3,4,0.75
354.65,3,3,1.00
139.60,3,3,1.00
...,...,...,...
200.08,0,1,0.00
200.10,0,2,0.00
200.11,0,1,0.00


Из этих данных сделать какие-то вы воды на основании суммы транзакции не получится.

Напишем функции, которые считают оцнку риска для клиена, продавца и категории, а также функцию, которая принимает решение о том, мошенническая ли транзакция, на основании суммы рисков. 

Мошенническими считаются транзакции с суммой рисков больше 2.

In [ ]:
def customer_evaluate(customer_id):
  if customer_id in customer_highrisk:
    return 3
  elif customer_id in customer_medrisk:
    return 2
  elif customer_id in customer_lowrisk:
    return 1
  else:
    return 0

In [ ]:
def merchant_evaluate(merchant_id):
  if merchant_id in merchant_highrisk:
    return 3
  elif merchant_id in merchant_medrisk:
    return 2
  elif merchant_id in merchant_lowrisk:
    return 1
  else:
    return 0

In [ ]:
def category_evaluate(category_id):
  if category_id in category_highrisk:
    return 3
  elif category_id in category_medrisk:
    return 2
  elif category_id in category_lowrisk:
    return 1
  else:
    return 0

In [ ]:
def detect_fraud(row):
  sum = customer_evaluate(row['customer']) + merchant_evaluate(row['merchant']) + category_evaluate(row['category'])
  if sum > 2:
    return 1
  else:
    return 0

In [ ]:
eval_data['fraud_prediction'] = eval_data.apply(detect_fraud, axis=1)

Оценим получившийся результат с помощью метрик  precision, recall и f1_score.

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
alg_precision = precision_score(eval_data['fraud'], eval_data['fraud_prediction'])
alg_recall = recall_score(eval_data['fraud'], eval_data['fraud_prediction'])
alg_f1_score = f1_score(eval_data['fraud'], eval_data['fraud_prediction'])
print(
    'Precision', alg_precision, '\nRecall', alg_recall, '\nF1 score', alg_f1_score
)

Precision 0.38405797101449274 
Recall 0.9464285714285714 
F1 score 0.5463917525773196


По метрикам видно, что такой алгоритм хорошо находит настоящие мошеннические транзакции, но у него также много ложноположительных результатов, то есть он считает большинство транзакций мошенническими; клиентов такой подход не обрадует).

Попробуем изменить параметры, по которым рассчитывается результат (изменим значение суммы с 2 до 3), и посчитаем метрики.

In [ ]:
def detect_fraud_2(row):
  sum = customer_evaluate(row['customer']) + merchant_evaluate(row['merchant']) + category_evaluate(row['category'])
  if sum > 3:
    return 1
  else:
    return 0

In [ ]:
eval_data['fraud_prediction_2'] = eval_data.apply(detect_fraud_2, axis=1)

In [ ]:
alg_precision_2 = precision_score(eval_data['fraud'], eval_data['fraud_prediction_2'])
alg_recall_2 = recall_score(eval_data['fraud'], eval_data['fraud_prediction_2'])
alg_f1_score_2 = f1_score(eval_data['fraud'], eval_data['fraud_prediction_2'])

Составим таблицу, куда будем записывать метрики всех моделей.

In [ ]:
metrics = pd.DataFrame()
metrics['metrics'] = ['precision', 'recall', 'f1_score']
metrics.set_index('metrics', inplace=True)
metrics['Algorythm (sum > 3)'] = [alg_precision_2, alg_recall_2, alg_f1_score_2]
metrics

,Algorythm (sum > 3)
metrics,
precision,0.623377
recall,0.857143
f1_score,0.721805


Как и ожидалось, полнота уменьшилась, а точность возросла почти в два раза. Оставим такие метрики в качестве baseline.

Для обучения модели подготовим данные и разделим на обучающую и валидационную выборки.

In [ ]:
data.head()

,step,customer,age,gender,merchant,category,amount,fraud
0,0,C1093826151,4,M,M348934600,es_transportation,4.55,0
1,0,C352968107,2,M,M348934600,es_transportation,39.68,0
2,0,C2054744914,4,F,M1823072687,es_transportation,26.89,0
3,0,C1760612790,3,M,M348934600,es_transportation,17.25,0
4,0,C757503768,5,M,M348934600,es_transportation,35.72,0


In [ ]:
data['age'].replace('U', 7, inplace=True)

In [ ]:
data['age'] = data['age'].astype('int32')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594643 entries, 0 to 594642
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   step      594643 non-null  int64  
 1   customer  594643 non-null  object 
 2   age       594643 non-null  int32  
 3   gender    594643 non-null  object 
 4   merchant  594643 non-null  object 
 5   category  594643 non-null  object 
 6   amount    594643 non-null  float64
 7   fraud     594643 non-null  int64  
dtypes: float64(1), int32(1), int64(2), object(4)
memory usage: 34.0+ MB


Применим функцию, которая кодирует числовыми значениями категориальные признаки.

In [ ]:
from sklearn import preprocessing

def number_encode_features(init_df):
    result = init_df.copy()
    encoders = {}
    for column in result.columns:
        if result.dtypes[column] == np.object:
            encoders[column] = preprocessing.LabelEncoder()
            result[column] = encoders[column].fit_transform(result[column])
    return result, encoders

In [ ]:
encoded_data, encoders = number_encode_features(data)
encoded_data.head() 

,step,customer,age,gender,merchant,category,amount,fraud
0,0,210,4,2,30,12,4.55,0
1,0,2753,2,2,30,12,39.68,0
2,0,2285,4,1,18,12,26.89,0
3,0,1650,3,2,30,12,17.25,0
4,0,3585,5,2,30,12,35.72,0


In [ ]:
y = encoded_data['fraud']
train_data = encoded_data.drop(columns='fraud')

In [ ]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(train_data, y)

Попробуем обучить модели из следущего списка; посмотрим, какие будут метрики при стандартных параметрах.

Модели:

* Logistic Regression
* Naive Bayes
* Decision Tree
* Support Vector Machines

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC

In [ ]:
models = {
    'Logistic Regression': LogisticRegression(),
    'Naive Bayes': MultinomialNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'Support Vector Machines': LinearSVC()
}

In [ ]:
from tqdm import tqdm

for model_name in tqdm(models.keys()):
  model = models[model_name]
  model.fit(train_X, train_y)
  predictions = model.predict(val_X)
  model_precision = precision_score(val_y, predictions)
  model_recall = recall_score(val_y, predictions)
  model_f1_score = f1_score(val_y, predictions)
  metrics[model_name] = [model_precision, model_recall, model_f1_score]


  0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)

 75%|███████▌  | 3/4 [00:08<00:03,  3.93s/it]/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)

100%|██████████| 4/4 [02:02<00:00, 30.71s/it]


Посмотрим на получившиеся метрики.

In [ ]:
metrics

,Algorythm (sum > 3),Logistic Regression,Naive Bayes,Decision Tree,Support Vector Machines
metrics,,,,,
precision,0.623377,0.801413,0.124723,0.762486,0.933265
recall,0.857143,0.546867,0.723621,0.735940,0.486877
f1_score,0.721805,0.650111,0.212773,0.748978,0.639916


Попробуем подобрать параметры, используя GridSearchCV.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'Logistic Regression': {
        'penalty' : ['l1', 'l2'],
        'solver' : ['liblinear', 'saga'],
        'max_iter': [100, 300, 500, 700, 1000]
    },
    'Naive Bayes': {
        'fit_prior': [False, True]
    },
    'Decision Tree': {
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 50, 100, 150],
        'min_samples_split': np.linspace(2, 5, 4, dtype='int'),
        'min_samples_leaf': np.linspace(1, 3, 3, dtype='int')
    },
    'Support Vector Machines': {
        'loss': ['hinge', 'squared_hinge'],
        'max_iter': [1000, 2000]
    }
}

best_params = dict()

for model_name in models.keys():
  model = models[model_name]
  model_grid = param_grid[model_name]
  gs = GridSearchCV(model, model_grid, return_train_score=True, n_jobs=-1, verbose=True)
  gs.fit(train_X, train_y)
  best_params[model_name] = gs.best_params_


Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 54.7min finished


Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.4s finished


Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   44.7s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  6.1min finished


Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 34.3min finished
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
best_params

{'Decision Tree': {'criterion': 'gini',
  'max_depth': None,
  'min_samples_leaf': 2,
  'min_samples_split': 5},
 'Logistic Regression': {'max_iter': 100,
  'penalty': 'l1',
  'solver': 'liblinear'},
 'Naive Bayes': {'fit_prior': True},
 'Support Vector Machines': {'loss': 'squared_hinge', 'max_iter': 2000}}

In [ ]:
models_cv = {
    'Logistic Regression': LogisticRegression(max_iter=100, penalty='l1', solver='liblinear'),
    'Naive Bayes': MultinomialNB(fit_prior=True),
    'Decision Tree': DecisionTreeClassifier(criterion='gini', max_depth=None, min_samples_leaf=2, min_samples_split=5),
    'Support Vector Machines': LinearSVC(loss='squared_hinge', max_iter=2000)
}

for model_name in tqdm(models_cv.keys()):
  model = models_cv[model_name]
  model.fit(train_X, train_y)
  predictions = model.predict(val_X)
  model_precision = precision_score(val_y, predictions)
  model_recall = recall_score(val_y, predictions)
  model_f1_score = f1_score(val_y, predictions)
  metrics[model_name + ' CV'] = [model_precision, model_recall, model_f1_score]


 75%|███████▌  | 3/4 [00:06<00:02,  2.66s/it]/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)

100%|██████████| 4/4 [03:56<00:00, 59.04s/it]


In [ ]:
metrics.T.sort_values('recall', ascending=False)

metrics,precision,recall,f1_score
Algorythm (sum > 3),0.623377,0.857143,0.721805
Decision Tree,0.762486,0.735940,0.748978
Decision Tree CV,0.803444,0.724692,0.762039
Naive Bayes,0.124723,0.723621,0.212773
Naive Bayes CV,0.124723,0.723621,0.212773
Support Vector Machines CV,0.791333,0.635779,0.705079
Logistic Regression CV,0.881131,0.567756,0.690554
Logistic Regression,0.801413,0.546867,0.650111
Support Vector Machines,0.933265,0.486877,0.639916


По сравнению с baseline у всех моделей слишком низкая полнота.

Попробуем применить метод SMOTE для балансировки выборки.

In [ ]:
train_X.shape, train_y.shape

((445982, 7), (445982,))

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(ratio='minority')
X_sm, y_sm = smote.fit_sample(train_X, train_y)

X_sm.shape, y_sm.shape

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


((881298, 7), (881298,))

In [ ]:
for model_name in tqdm(models_cv.keys()):
  model = models_cv[model_name]
  model.fit(X_sm, y_sm)
  predictions = model.predict(val_X)
  model_precision = precision_score(val_y, predictions)
  model_recall = recall_score(val_y, predictions)
  model_f1_score = f1_score(val_y, predictions)
  metrics[model_name + ' SMOTE'] = [model_precision, model_recall, model_f1_score]


 75%|███████▌  | 3/4 [00:17<00:06,  6.45s/it]/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)

100%|██████████| 4/4 [08:55<00:00, 133.76s/it]


In [ ]:
metrics.T.sort_values('f1_score', ascending=False)

metrics,precision,recall,f1_score
Decision Tree SMOTE,0.787948,0.742367,0.764479
Decision Tree CV,0.803444,0.724692,0.762039
Decision Tree,0.762486,0.735940,0.748978
Algorythm (sum > 3),0.623377,0.857143,0.721805
Support Vector Machines CV,0.791333,0.635779,0.705079
Logistic Regression CV,0.881131,0.567756,0.690554
Logistic Regression,0.801413,0.546867,0.650111
Support Vector Machines,0.933265,0.486877,0.639916
Logistic Regression SMOTE,0.172286,0.870380,0.287636
Naive Bayes,0.124723,0.723621,0.212773


По этим метрикам лидирует модель DecisionTree. 

Попробуем использовать модель RandomForest для улучшения результатов.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_rfc = RandomForestClassifier(criterion='entropy', n_estimators=150, min_samples_split=3, n_jobs=-1)
model_rfc.fit(X_sm, y_sm)
predictions_rfc = model_rfc.predict(val_X)
model_rfc_precision = precision_score(val_y, predictions_rfc)
model_rfc_recall = recall_score(val_y, predictions_rfc)
model_rfc_f1_score = f1_score(val_y, predictions_rfc)
metrics['RandomForest SMOTE'] = [model_rfc_precision, model_rfc_recall, model_rfc_f1_score]

In [ ]:
metrics.T.sort_values('f1_score',ascending=False)

metrics,precision,recall,f1_score
RandomForest SMOTE,0.825549,0.785753,0.805159
Decision Tree SMOTE,0.787948,0.742367,0.764479
Decision Tree CV,0.803444,0.724692,0.762039
Decision Tree,0.762486,0.735940,0.748978
Algorythm (sum > 3),0.623377,0.857143,0.721805
Support Vector Machines CV,0.791333,0.635779,0.705079
Logistic Regression CV,0.881131,0.567756,0.690554
Logistic Regression,0.801413,0.546867,0.650111
Support Vector Machines,0.933265,0.486877,0.639916
Logistic Regression SMOTE,0.172286,0.870380,0.287636


По этим метрикам самой приемлемой моделью является RandomForest, обученный на сбалансированной выборке. У других моделей, у которых полнота выше, сильно страдает точность.

Подготовим данные к загрузке в базу: добавим к тестовой выборке предсказания модели RandomForest.

Напишем функцию для обратного преобразования закодированных данных.

In [ ]:
def number_decode_features(init_df):
    result = init_df.copy()
    for column in result.columns:
        if column in encoders:
            result[column] = encoders[column].inverse_transform(result[column])
    return result

In [ ]:
db_data = number_decode_features(val_X)
db_data['fraud_prediction'] = predictions_rfc
db_data.head()

,step,customer,age,gender,merchant,category,amount,fraud_prediction
592449,179,C1648096390,5,M,M348934600,es_transportation,20.32,0
85693,31,C747175090,5,F,M348934600,es_transportation,4.45,0
490707,151,C1386324281,2,M,M85975013,es_food,5.56,0
126896,45,C626467543,4,F,M1823072687,es_transportation,13.38,0
205446,70,C1986701933,3,F,M855959430,es_hyper,15.98,0


In [ ]:
db_data['age'] = db_data.astype(np.object)
db_data['age'].replace(7, 'U', inplace=True)

In [ ]:
db_data = db_data.sort_index()
db_data

,step,customer,age,gender,merchant,category,amount,fraud_prediction
1,0,C352968107,0,M,M348934600,es_transportation,39.68,0
4,0,C757503768,0,M,M348934600,es_transportation,35.72,0
8,0,C105845174,0,M,M348934600,es_transportation,32.40,0
12,0,C623601481,0,M,M50039827,es_health,68.79,0
13,0,C1865204568,0,M,M1823072687,es_transportation,20.32,0
...,...,...,...,...,...,...,...,...
594618,179,C1088853847,179,F,M151143676,es_barsandrestaurants,4.11,0
594627,179,C1007572087,179,F,M1823072687,es_transportation,35.30,0
594629,179,C1990073844,179,M,M1823072687,es_transportation,29.37,0
594637,179,C748358246,179,M,M1823072687,es_transportation,51.17,0


Подключимся к базе данных и загрузим в неё тестовую выборку.

In [ ]:
import psycopg2

DB_HOST = '89.223.95.138'
DB_USER = 'student_16'
DB_USER_PASSWORD = 'student_16'
DB_NAME = 'student_16'

conn = psycopg2.connect(host=DB_HOST, user=DB_USER, password=DB_USER_PASSWORD, dbname=DB_NAME)
cursor = conn.cursor()

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
pd.read_sql_query("""
SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND 
    schemaname != 'information_schema'
    """, conn)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,adv_glossary,student_16,None,True,False,True,False
1,public,adv_events,student_16,None,True,False,True,False


Создадим в базе данных таблицу fraud_detection.

In [ ]:
query = """
  CREATE TABLE IF NOT EXISTS fraud_detection (
    id SERIAL PRIMARY KEY,
    step INT,
    customer VARCHAR,
    age	VARCHAR,
    gender	VARCHAR,
    merchant	VARCHAR,
    category	VARCHAR,
    amount	FLOAT,
    fraud_prediction INT
  )
"""
cursor.execute(query)
conn.commit()

Загрузим данные из тестовой выборки в базу данных.

In [ ]:
for i in tqdm(range(10000, 150000, 10000)):
  for idx, row in db_data.iloc[i - 10000:i].iterrows():
    cursor.execute("""
    INSERT INTO fraud_detection
    (step, customer, age, gender, merchant, category, amount, fraud_prediction)
    VALUES {}
    """.format(tuple(row)))

  conn.commit()


100%|██████████| 14/14 [5:09:42<00:00, 1327.30s/it]


In [ ]:
for idx, row in db_data.iloc[140000:].iterrows():
  cursor.execute("""
  INSERT INTO fraud_detection
  (step, customer, age, gender, merchant, category, amount, fraud_prediction)
  VALUES {}
  """.format(tuple(row)))

conn.commit()

Проверим, что в данные в базу данных загрузились правильно и количество записей соответствует.

In [ ]:
pd.read_sql_query("""
SELECT * FROM fraud_detection LIMIT 5
    """, conn)

,id,step,customer,age,gender,merchant,category,amount,fraud_prediction
0,1,0,C352968107,0,M,M348934600,es_transportation,39.68,0
1,2,0,C757503768,0,M,M348934600,es_transportation,35.72,0
2,3,0,C105845174,0,M,M348934600,es_transportation,32.40,0
3,4,0,C623601481,0,M,M50039827,es_health,68.79,0
4,5,0,C1865204568,0,M,M1823072687,es_transportation,20.32,0


In [ ]:
pd.read_sql_query("""
SELECT count(*) FROM fraud_detection
    """, conn)

,count
0,148661
